In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
with open("/kaggle/working/TraningConfig.py", "w") as f:
    f.write("""
from pathlib import Path

class TrainingConfig:
    '''
    All configurations required for the models to train
    '''

    DATA_DIR = Path('output/data')
    OUTPUT_DIR = Path('output')
    MODEL_DIR = OUTPUT_DIR / "models_greatlakes"
    VIZ_DIR = OUTPUT_DIR / "visualizations"

    #? Model Hyperparameters - Enhanced for better performance
    EMBEDDING_DIM = 256
    HIDDEN_DIM = 512
    NUM_HEADS = 8
    NUM_LAYERS = 4
    DROPOUT = 0.3
    MAX_SEQ_LENGTH = 50

    #? Training hyperparameters - Optimized
    BATCH_SIZE = 128
    LEARNING_RATE = 0.0005
    NUM_EPOCHS = 5
    PATIENCE = 8
    WARMUP_EPOCHS = 3
    LABEL_SMOOTHING = 0.1

    TOP_K_VALUES = [1, 5, 10, 20]

    #? System
    NUM_WORKERS = 4
    RANDOM_SEED = 42

    def __init__(self):
        '''Initialize output directories'''
        self.MODEL_DIR.mkdir(parents=True, exist_ok=True)
        self.VIZ_DIR.mkdir(parents=True, exist_ok=True)
        self.DATA_DIR.mkdir(parents=True, exist_ok=True)
""")


In [ ]:
with open("/kaggle/working/config.py", "w") as f:
    f.write("""
import json
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict, Counter
from typing import List, Dict, Tuple
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# For ML models
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# For visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# For Arrow file export
import pyarrow as pa
import pyarrow.parquet as pq



class Config:
    '''
    This class holds all the configurations like 
    input file path, output file path and hyperparameters for the models
    '''
    MPD_DIR = Path('/kaggle/input/millionplaylistdataset/data')
    OUTPUT_DIR = Path('output')
    HF_DATASET_PATH = "/kaggle/input/mpd-audio-features/extracted_audio_features.csv"
    NUM_FILES_TO_PROCESS = 100  #? How many files should we process in the dataset
    MIN_PLAYLIST_LENGTH = 5 #? Lower bound for number of songs in the playlist
    MAX_PLAYLIST_LENGTH = 200  #? Upper bound for number of songs in the playlist
    MIN_SONG_FREQUENCY = 10  #? How many times that song should be present in all playlists in order to be used.

    AUDIO_FEATURES = ['danceability', 'energy', 'loudness', 'speechiness', 
                     'acousticness', 'instrumentalness', 'liveness', 
                     'valence', 'tempo', 'duration_ms']
    
    KNN_NEIGHBORS = 20 #? Hyperparameter for NearestNeigbor classifier
    RANDOM_SEED = 42 #? For reproduciblity of code
    
    # Data splits
    TRAIN_RATIO = 0.70 #? 70% of data for training
    VAL_RATIO = 0.15 #? 15% Data for validation
    TEST_RATIO = 0.15 #? 15% data for testing
    
    # Evaluation
    TOP_K_VALUES = [1, 5, 10, 20]
    
    def __init__(self):
        self.OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        (self.OUTPUT_DIR / "models").mkdir(exist_ok=True)
        (self.OUTPUT_DIR / "visualizations").mkdir(exist_ok=True)
        (self.OUTPUT_DIR / "data").mkdir(exist_ok=True)
        (self.OUTPUT_DIR / "metrics").mkdir(exist_ok=True)

    
""")


In [ ]:
with open("/kaggle/working/dataloader.py", "w") as f:
    f.write("""
from config import Config
from typing import List, Dict
from tqdm import tqdm
import json
import pandas as pd
from collections import Counter

class SpotifyDataLoader:
    '''
    Handles loading dataset and merging spotify MPD audio features
    '''
    def __init__(self, config: Config):
        self.config = config
        self.playlists = []
        self.track_uri_to_id = {}
        self.audio_features_df = None

    def load_mpd_files(self) -> List[Dict]:
        '''
        Loading the Spotify MPD Dataset
        '''
        print('[1/7] Loading the spotify MPD Dataset')
        
        json_files = sorted(list(self.config.MPD_DIR.glob("*.json")))

        if len(json_files) == 0:
            raise FileNotFoundError(f'No JSON files found in {self.config.MPD_DIR}')
        
        files_to_load = json_files[:self.config.NUM_FILES_TO_PROCESS]

        print(f'Found {len(json_files)} files, loading {len(files_to_load)} files...')

        all_playlists = []

        for json_file in tqdm(files_to_load, desc="Loading JSON files"):
            with open(json_file, 'r') as f:
                data = json.load(f)
                all_playlists.extend(data['playlists'])
        
        print(f"Loaded {len(all_playlists):,} playlists")
        return all_playlists
    

    def load_audio_features(self) -> pd.DataFrame:
        df = pd.read_csv(self.config.HF_DATASET_PATH)

        print(f"Loaded {len(df):,} tracks with audio features")
        print(f"Columns: {df.columns.tolist()}")
        
        return df
    

    def preprocess_data(self, playlists: List[Dict], audio_df: pd.DataFrame):
        '''
        Clean and merge playlist data with audio features
        '''

        filtered_playlists = [
            p for p in playlists 
            if self.config.MIN_PLAYLIST_LENGTH <= len(p['tracks']) <= self.config.MAX_PLAYLIST_LENGTH
        ]

        print(f"After length filtering: {len(filtered_playlists):,} playlists")

        # Count track frequency
        track_counter = Counter()
        for playlist in filtered_playlists:
            for track in playlist['tracks']:
                track_counter[track['track_uri']] += 1

        frequent_tracks = {
            uri for uri, count in track_counter.items()
            if count >= self.config.MIN_SONG_FREQUENCY
        }
        print(f"Tracks appearing in {self.config.MIN_SONG_FREQUENCY}+ playlists: {len(frequent_tracks):,}")

        print("Building track URI to ID mapping...")
        if 'track_uri' in audio_df.columns and 'id' not in audio_df.columns:
            audio_df = audio_df.rename(columns={'track_uri': 'id'})
        
        uri_to_id = {}
        for _, row in tqdm(audio_df.iterrows(), total=len(audio_df), desc="Mapping URIs"):
            if 'id' in row and pd.notna(row['id']):
                uri = f"spotify:track:{row['id']}"
                uri_to_id[uri] = row['id']
        
        print(f"Mapped {len(uri_to_id):,} track URIs")

        final_playlists = []
        for playlist in tqdm(filtered_playlists, desc="Filtering tracks"):
            filtered_tracks = [
                t for t in playlist['tracks']
                if t['track_uri'] in uri_to_id and t['track_uri'] in frequent_tracks
            ]
            if len(filtered_tracks) >= self.config.MIN_PLAYLIST_LENGTH:
                playlist['tracks'] = filtered_tracks
                final_playlists.append(playlist)
        
        print(f"Final dataset: {len(final_playlists):,} playlists")

        final_track_uris = set()
        for playlist in final_playlists:
            for track in playlist['tracks']:
                final_track_uris.add(track['track_uri'])

        print(f"Final vocabulary: {len(final_track_uris):,} unique tracks")

        final_track_ids = [
            uri_to_id[uri] for uri in final_track_uris if uri in uri_to_id
        ]

        audio_df_filtered = audio_df[audio_df['id'].isin(final_track_ids)].copy()

        print(f"Audio features for {len(audio_df_filtered):,} tracks")

        self.playlists = final_playlists
        self.track_uri_to_id = uri_to_id
        self.audio_features_df = audio_df_filtered

        return final_playlists, audio_df_filtered, uri_to_id
""")


NO need to install package. Already available!!

In [ ]:
!python /kaggle/working/main.py


STARTING ML PIPELINE

[1/7] Loading the spotify MPD Dataset
Found 1000 files, loading 100 files...
Loading JSON files: 100%|█████████████████████| 100/100 [00:36<00:00,  2.74it/s]
Loaded 100,000 playlists
Loaded 2,262,292 tracks with audio features
Columns: ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', 'track_uri']
After length filtering: 96,507 playlists
Tracks appearing in 10+ playlists: 67,345
Building track URI to ID mapping...
Mapping URIs: 100%|████████████████| 2262292/2262292 [01:29<00:00, 25145.37it/s]
Mapped 2,262,292 track URIs
Filtering tracks: 100%|████████████████| 96507/96507 [00:05<00:00, 18659.37it/s]
Final dataset: 92,991 playlists
Final vocabulary: 67,345 unique tracks
Audio features for 67,345 tracks

[4/7] Creating audio feature embeddings...
Using features: ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalne

In [ ]:
!python /kaggle/working/main_2.py

DEEP LEARNING TRAINING

[1/5] Building vocabulary...
Vocabulary size: 67,345 tracks

[2/5] Creating datasets...
Train: 3,293,908 sequences
Val: 707,739 sequences
Test: 702,677 sequences

DATASET DIAGNOSTICS

Vocabulary size: 67,345
Dataset expects num_items: 67,347
Match: ✅

Sample training example:
  History indices: [4255]... (first 5)
  Target index: 54293
  Sequence length: 1

🔍 Critical Checks:
  Max history index: 66986
  Max target index: 66986
  Num items in model: 67347
  Max index found: 67328
  Should be < 67347: ✅
  Samples with UNK tokens: 0/1000 (0.0%)


[3/5] Training GRU...

Training GRU

Warmup Epoch 1/3 - LR: 0.000167

POST-EPOCH-1 DIAGNOSTICS

Embedding Statistics:
  Mean norm: 15.9827
  Std norm: 0.7089
  Min norm: 13.1815
  Max norm: 19.2997

Prediction Diversity:
  Entropy: 11.0911 / 11.1176 (99.8%)


Epoch 1/50
Evaluating GRU: 100%|███████████████████| 5530/5530 [01:31<00:00, 60.49it/s]
Train Loss: 9.6372
Val Loss: 9.1404
Val Top-1 Acc: 0.49%
Val Top-5 Acc: 2.02%

ran the above code for 3hrs

In [ ]:
!python /kaggle/working/main_2.py

DEEP LEARNING TRAINING

[1/5] Building vocabulary...
Vocabulary size: 67,345 tracks

[2/5] Creating datasets...
Train: 3,293,908 sequences
Val: 707,739 sequences
Test: 702,677 sequences

DATASET DIAGNOSTICS

Vocabulary size: 67,345
Dataset expects num_items: 67,347
Match: ✅

Sample training example:
  History indices: [4255]... (first 5)
  Target index: 54293
  Sequence length: 1

🔍 Critical Checks:
  Max history index: 66986
  Max target index: 66986
  Num items in model: 67347
  Max index found: 67328
  Should be < 67347: ✅
  Samples with UNK tokens: 0/1000 (0.0%)


[3/5] Training GRU...

Training GRU

Warmup Epoch 1/3 - LR: 0.001667

POST-EPOCH-1 DIAGNOSTICS

Embedding Statistics:
  Mean norm: 15.9827
  Std norm: 0.7089
  Min norm: 13.1815
  Max norm: 19.2997

Prediction Diversity:
  Entropy: 11.0911 / 11.1176 (99.8%)


Epoch 1/5
Evaluating GRU: 100%|███████████████████| 5530/5530 [01:32<00:00, 59.53it/s]
Train Loss: 9.8191
Val Loss: 9.3473
Val Top-1 Acc: 0.32%
Val Top-5 Acc: 1.41%


In [ ]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict, Counter
from typing import List, Dict, Tuple
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# For ML models
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# For visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# For Arrow file export
import pyarrow as pa
import pyarrow.parquet as pq





def load_mpd_files():
    '''
    Loading the Spotify MPD Dataset
    '''
    print('\n[1/7] Loading the spotify MPD Dataset')
    
    json_files = sorted(list(Path('/kaggle/input/millionplaylistdataset/data').glob("*.json"))) #* Find all the files that are of the type ".json"

    if len(json_files) == 0:
        #! Raise error if file is not found for preprocessing
        raise FileNotFoundError(f'No JSON files are found in {self.config.MPD_DIR}')
    
    files_to_load = json_files[:100]

    print(f'Found items {len(json_files)}, loading {len(files_to_load)} files..')

    all_playlists = []

    for json_file in tqdm(files_to_load, desc="Loading JSON files"):
        with open(json_file, 'r') as f: #TODO: Open each and every file in the path given and load it.
            data = json.load(f)
            all_playlists.extend(data['playlists']) #TODO: Extend the playlist in the all_playlists 
    
    print(f"Loaded {len(all_playlists):,} playlists")
    return all_playlists

In [28]:
playlists = load_mpd_files()
type(playlists)


[1/7] Loading the spotify MPD Dataset
Found items 1000, loading 100 files..


Loading JSON files: 100%|██████████| 100/100 [00:27<00:00,  3.61it/s]


Loaded 100,000 playlists


list

In [41]:
audio_df = pd.read_csv("/kaggle/input/mpd-audio-features/extracted_audio_features.csv")

In [45]:
audio_df['track_uri'].duplicated().sum()

0

In [43]:
audio_df

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,track_uri
0,0.160,0.0000,83856.0,0.04030,0.998000,1.0,0.9300,-21.209,1.0,0.0000,0.000,0.0,0.0000,423KoEUpycHLNi4vesOPpT
1,0.976,0.2880,243840.0,0.16600,0.874000,5.0,0.0917,-19.562,1.0,0.0330,149.746,4.0,0.2800,2twQjIxyECbDWZroaALoXW
2,0.323,0.0955,49773.0,0.42000,0.967000,7.0,0.2510,-10.077,1.0,0.0410,165.260,4.0,0.2240,4ekaz63eTiJmxVhxr8dm42
3,0.986,0.3410,194440.0,0.11200,0.920000,0.0,0.0987,-16.130,1.0,0.0319,74.386,4.0,0.2650,5ixFH1Eqdx25tiIGoYEuzD
4,0.978,0.2700,129133.0,0.00377,0.903000,5.0,0.0762,-27.014,1.0,0.0471,106.242,4.0,0.0538,77kzsKzeF53njh9vQ2f8Ct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2262287,0.215,0.5650,225280.0,0.62200,0.000603,7.0,0.1280,-8.176,1.0,0.0470,120.007,4.0,0.4660,7DnIiruL3WwET6fM0EE80w
2262288,0.215,0.5650,225280.0,0.62200,0.000603,7.0,0.1280,-8.176,1.0,0.0470,120.007,4.0,0.4660,5rG8C2j2GssqEqobZ38sCq
2262289,0.215,0.5650,225280.0,0.62200,0.000603,7.0,0.1280,-8.176,1.0,0.0470,120.007,4.0,0.4660,6ZrMSD58Au73IdqJyvicin
2262290,0.215,0.5650,225280.0,0.62200,0.000603,7.0,0.1280,-8.176,1.0,0.0470,120.007,4.0,0.4660,65qH0fE7j8MrtvWFnBmm87


In [46]:
filtered_playlists = [p for p in playlists if 5 <= len(p['tracks']) <= 200]

print(f"After length filtering: {len(filtered_playlists):,} playlists")


After length filtering: 96,507 playlists


In [48]:
unique_tracks = {
    t['track_uri']
    for p in filtered_playlists
    for t in p['tracks']
}
print("Unique track URIs:", len(unique_tracks))


Unique track URIs: 627211


In [33]:
# Extract all unique track URIs and count frequencies
track_counter = Counter()
for playlist in filtered_playlists:
    for track in playlist['tracks']:
        track_counter[track['track_uri']] += 1


frequent_tracks = {uri for uri, count in track_counter.items() 
                  if count >= 10}
print(f"Tracks appearing in {10}+ playlists: {len(frequent_tracks):,}")


Tracks appearing in 10+ playlists: 67,345


In [38]:
audio_ids = set(audio_df['id'].astype(str))
print("Audio feature IDs:", len(audio_ids))


Audio feature IDs: 2262292


In [39]:
mpd_ids = set(uri.split(":")[-1] for uri in frequent_tracks)
print("MPD frequent track IDs:", len(mpd_ids))


MPD frequent track IDs: 67345


In [40]:
common_ids = mpd_ids & audio_ids
print("Overlap (shared track IDs):", len(common_ids))


Overlap (shared track IDs): 3


In [11]:

print("Building track URI to ID mapping...")
if 'track_uri' in audio_df.columns and 'id' not in audio_df.columns:
    audio_df = audio_df.rename(columns={'track_uri': 'id'})

uri_to_id = {}
for _, row in tqdm(audio_df.iterrows(), total=len(audio_df), desc="Mapping URIs"):
    if 'id' in row and pd.notna(row['id']):
        uri = f"spotify:track:{row['id']}"
        uri_to_id[uri] = row['id']

print(f"Successfully mapped {len(uri_to_id):,} track URIs")


Building track URI to ID mapping...


Mapping URIs: 100%|██████████| 2262292/2262292 [01:54<00:00, 19777.63it/s]

Successfully mapped 2,262,292 track URIs


In [21]:
common_uris = set(uri_to_id.keys()).intersection(frequent_tracks)
print(f"Number of common track URIs: {len(common_uris)}")


Number of common track URIs: 3


In [23]:
frequent_tracks
# 'spotify:track:1TQKEwq4y9SkNciJuisE1m',

uri_to_id
# 'spotify:track:423KoEUpycHLNi4vesOPpT': '423KoEUpycHLNi4vesOPpT',

{'spotify:track:423KoEUpycHLNi4vesOPpT': '423KoEUpycHLNi4vesOPpT',
 'spotify:track:2twQjIxyECbDWZroaALoXW': '2twQjIxyECbDWZroaALoXW',
 'spotify:track:4ekaz63eTiJmxVhxr8dm42': '4ekaz63eTiJmxVhxr8dm42',
 'spotify:track:5ixFH1Eqdx25tiIGoYEuzD': '5ixFH1Eqdx25tiIGoYEuzD',
 'spotify:track:77kzsKzeF53njh9vQ2f8Ct': '77kzsKzeF53njh9vQ2f8Ct',
 'spotify:track:5qT7hy6SYuGtWGGlXZ6HZO': '5qT7hy6SYuGtWGGlXZ6HZO',
 'spotify:track:0EsSyhD4Jo0mOBFAxYcHdy': '0EsSyhD4Jo0mOBFAxYcHdy',
 'spotify:track:5P8az4YEo8XoFjqToGjbBe': '5P8az4YEo8XoFjqToGjbBe',
 'spotify:track:536AgB3m3BviF6GaUsRws1': '536AgB3m3BviF6GaUsRws1',
 'spotify:track:6G4RYeUMEOFgi5XRqveZZZ': '6G4RYeUMEOFgi5XRqveZZZ',
 'spotify:track:0l9ieF8fr79bvf2FYfRVVf': '0l9ieF8fr79bvf2FYfRVVf',
 'spotify:track:19IxmsuxDDr2WVHgPTwMuW': '19IxmsuxDDr2WVHgPTwMuW',
 'spotify:track:7AwiTLNZuYKAtVvawiXou5': '7AwiTLNZuYKAtVvawiXou5',
 'spotify:track:3AYBKkoSM1R0w3qJCtXRsP': '3AYBKkoSM1R0w3qJCtXRsP',
 'spotify:track:3oSOAbPQzlQHn2Q80VDqjA': '3oSOAbPQzlQHn2Q80VDq

In [24]:
print("Sample MPD track_uri:", next(iter(track_counter)))
print("Sample audio_df id:", audio_df['id'].iloc[0])
print("Sample uri_to_id key:", next(iter(uri_to_id)))


Sample MPD track_uri: spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
Sample audio_df id: 423KoEUpycHLNi4vesOPpT
Sample uri_to_id key: spotify:track:423KoEUpycHLNi4vesOPpT


In [25]:
print("Does MPD uri match the constructed one?")
sample = next(iter(track_counter))
print(sample in uri_to_id)


Does MPD uri match the constructed one?
False


In [19]:
final_playlists = []
for playlist in tqdm(filtered_playlists, desc="Filtering tracks"):
    filtered_tracks = [
        t for t in playlist['tracks'] 
        if t['track_uri'] in uri_to_id and t['track_uri'] in frequent_tracks
    ]
    
    if len(filtered_tracks) >= 2:
        playlist['tracks'] = filtered_tracks
        final_playlists.append(playlist)

print(f"Final dataset: {len(final_playlists):,} playlists")


Filtering tracks: 100%|██████████| 96507/96507 [00:00<00:00, 1013837.51it/s]

Final dataset: 3 playlists


In [ ]:

# Build final track vocabulary
final_track_uris = set()
for playlist in final_playlists:
    for track in playlist['tracks']:
        final_track_uris.add(track['track_uri'])

print(f"Final vocabulary: {len(final_track_uris):,} unique tracks")

# Filter audio features to only include tracks in vocabulary
final_track_ids = [uri_to_id[uri] for uri in final_track_uris if uri in uri_to_id]
audio_df_filtered = audio_df[audio_df['id'].isin(final_track_ids)].copy()

print(f"Audio features for {len(audio_df_filtered):,} tracks")

self.playlists = final_playlists
self.track_uri_to_id = uri_to_id
self.audio_features_df = audio_df_filtered